<a href="https://colab.research.google.com/github/HK-JUN/news_summary/blob/master/npr_news_searching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from transformers import pipeline,AutoTokenizer
from bs4 import BeautifulSoup
import datetime as dt

In [ ]:
START_DATE = "2024/11/30" #from START_DATE ~ now


def get_article_list(start_date):
  date_limit = dt.datetime.strptime(start_date,"%Y/%m/%d")
  today = dt.datetime.now()
  start=0
  continue_flag = True
  articles = []
  while continue_flag:
    url = f"https://www.npr.org/sections/business/archive?date={today.month}-{today.day}-{today.year}&count=100&start={start}"
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.content,"html.parser")
    items = soup.find_all("h2",attrs={"class":"title"})

    for item in items:
      content = item.find("a")
      title = content.get_text()
      link = content['href']
      issue_date = dt.datetime.strptime(item.parent.find("time")["datetime"],"%Y-%m-%d")
      if issue_date > date_limit:
        t = {"title":title,"url":link,"issue_date":issue_date}
        articles.append(t)
      else:
        continue_flag = False
    start +=100
  return articles

def get_content(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content,"html.parser")
  items = soup.find("div",attrs={"id":"storytext"}).find_all("p")
  contents = []
  for item in items:
    if item.parent['class'][0] != "caption":
      contents.append(item.get_text())
  result = "".join(contents)
  return result

def get_content(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content,"html.parser")
  items = soup.find("div",attrs={"id":"storytext"}).find_all("p")
  contents = []
  for item in items:
    if item.parent.has_attr("class"):
      if item.parent['class'][0] != "caption":
        contents.append(item.get_text())
  result = "".join(contents)
  return result

def get_overlapped_chunk(text,chunk,overlap):
  return [text[s:s+chunk] for s in range(0,len(text),chunk-overlap)]

summarizer = pipeline('summarization',model='google-t5/t5-small',device=0)
def summarize_article_chunk(article,chunk,overlap):
  outs = []
  for chunk in get_overlapped_chunk(article,chunk,overlap):
    out = summarizer(chunk,max_length=128,min_length=32)
    outs.append(out[0]['summary_text'])
  result = summarizer(''.join(outs),max_length=512,min_length=32)[0]['summary_text']
  return result

import torch
from transformers import AutoModel
from scipy.spatial.distance import cosine
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
  inputs = tokenizer(text,return_tensors='pt',max_length=650)
  with torch.no_grad():
    outputs = model(**inputs)
  return outputs.last_hidden_state[:,0,:].squeeze().numpy()

def compute_cos_sim(text1,text2):
  embedding1 = get_embedding(text1)
  embedding2 = get_embedding(text2)
  return 1 - cosine(embedding1,embedding2)

#--------
articles = get_article_list(START_DATE)
print(f"article {len(articles)} are found")
keyword = "bitcoin"
for i,article in enumerate(articles):
  summarized_result = summarize_article_chunk(get_content(article["url"]),1000,64)
  cos_score = compute_cos_sim(summarized_result,keyword)
  articles[i]['summary'] = summarized_result
  articles[i]['score'] = cos_score
  print(f"{i} completed")

sorted_list = sorted(articles,key=lambda x:x['score'],reverse=True)
for item in sorted_list[:5]:
  print(item)



Device set to use cpu


https://www.npr.org/sections/business/archive?date=2-16-2025&count=100&start=0
https://www.npr.org/sections/business/archive?date=2-16-2025&count=100&start=100
https://www.npr.org/sections/business/archive?date=2-16-2025&count=100&start=200
https://www.npr.org/sections/business/archive?date=2-16-2025&count=100&start=300
article 349 are found


Your max_length is set to 512, but your input_length is only 234. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=117)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0 completed


Your max_length is set to 128, but your input_length is only 10. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)
Your max_length is set to 512, but your input_length is only 476. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=238)


...

346 completed


Your max_length is set to 128, but your input_length is only 6. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=3)
Your max_length is set to 512, but your input_length is only 299. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=149)


347 completed


Your max_length is set to 512, but your input_length is only 102. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


348 completed
{'title': 'Bitcoin hits $100,000 for the first time. 3 things to know about an incredible ride', 'url': 'https://www.npr.org/2024/12/04/nx-s1-5202832/bitcoin-rally-100-000-crypto-trump', 'issue_date': datetime.datetime(2024, 12, 4, 0, 0), 'summary': "bitcoin hit the $100,000 milestone after Trump picked crypto supporter Paul Atkins as the next chair of the Securities and Exchange Commission earlier in the day .critics are raising alarms about bitcoin's spectacular ride . bitcoin has long had a history of fraud and scams . some critics believe they could even pose a risk to the country's financial system .atkins appointed to replace current SEC chair Gary Gensler .", 'score': 0.8993107057414577}
{'title': "Is this bitcoin's golden moment? These are 3 key things to watch for cryptos in 2025", 'url': 'https://www.npr.org/2025/01/06/nx-s1-5248284/bitcoin-rally-crypto-trump', 'issue_date': datetime.datetime(2025, 1, 6, 0, 0), 'summary': 'analysts believe bitcoin and other cryp

In [ ]:
url = "https://www.npr.org/2024/12/31/nx-s1-5243853/south-korea-boeing-airplane-crash"
def get_content(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.content,"html.parser")
  items = soup.find("div",attrs={"id":"storytext"}).find_all("p")
  contents = []
  for item in items:
    if item.parent['class'][0] != "caption":
      contents.append(item.get_text())
  result = "".join(contents)
  return result
article_result = get_content(url)

In [ ]:
type(article_result)
len(article_result.split(" "))

973

In [ ]:
summarizer = pipeline('summarization',model='google-t5/t5-small',device=0)
def get_overlapped_chunk(text,chunk,overlap):
  return [text[s:s+chunk] for s in range(0,len(text),chunk-overlap)]

def summarize_article_chunk(article,chunk,overlap):
  outs = []
  for chunk in get_overlapped_chunk(article,chunk,overlap):
    out = summarizer(chunk,max_length=128,min_length=32)
    outs.append(out[0]['summary_text'])
  result = summarizer(''.join(outs),max_length=512,min_length=32)[0]['summary_text']
  return result
summarized_result = summarize_article_chunk(article_result,1024,64)

Device set to use cpu
Your max_length is set to 128, but your input_length is only 107. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=53)
Your max_length is set to 512, but your input_length is only 410. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=205)


In [ ]:
import torch
from transformers import AutoModel
from scipy.spatial.distance import cosine
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
  inputs = tokenizer(text,return_tensors='pt',max_length=512)
  with torch.no_grad():
    outputs = model(**inputs)
  return outputs.last_hidden_state[:,0,:].squeeze().numpy()

def compute_cos_sim(text1,text2):
  embedding1 = get_embedding(text1)
  embedding2 = get_embedding(text2)
  return 1 - cosine(embedding1,embedding2)

compute_cos_sim(summarized_result,"festival in Korea")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


0.7202470799782853